# Phi-4 Mini ONNX Paralelné Volanie Funkcií - Návod

Tento notebook demonštruje, ako používať Phi-4 Mini s ONNX Runtime GenAI na paralelné volanie funkcií. Volanie funkcií umožňuje modelu inteligentne využívať externé nástroje a API na základe požiadaviek používateľa.

## Prehľad

V tomto návode sa naučíte:
- Nastaviť Phi-4 Mini s ONNX Runtime GenAI
- Definovať schémy funkcií na rezerváciu letov a hotelov
- Používať riadenú generáciu s gramatikou Lark pre štruktúrovaný výstup
- Vykonávať paralelné volania funkcií pre komplexné scenáre rezervácie cestovania

## Predpoklady

Pred spustením tohto notebooku sa uistite, že máte:
- Stiahnutý model Phi-4 Mini ONNX
- Nainštalovaný balík `onnxruntime-genai`
- Základné pochopenie konceptov volania funkcií


## Krok 1: Importovanie potrebných knižníc

Najskôr importujeme potrebné knižnice pre implementáciu volania funkcií.


In [1]:
import json

In [2]:
import onnxruntime_genai as og

## Krok 2: Nastavenie a konfigurácia modelu

Teraz nakonfigurujeme model Phi-4 Mini ONNX. Uistite sa, že cestu nahradíte skutočným adresárom vášho modelu.


In [ ]:
# TODO: Replace with your actual Phi-4 Mini ONNX model path
# Download from: https://huggingface.co/microsoft/Phi-4-mini-onnx
path = 'Your phi-4-mini-onnx path'  # Update this path!

In [4]:
config = og.Config(path)

In [5]:
model = og.Model(config)

In [6]:
tokenizer = og.Tokenizer(model)
tokenizer_stream = tokenizer.create_stream()

## Krok 3: Nastavenie parametrov generovania

Nastavte parametre generovania na ovládanie správania výstupu modelu. Tieto nastavenia zabezpečujú deterministické a cielené odpovede pre volanie funkcií.


In [7]:
# Configure generation parameters for deterministic function calling
search_options = {}
search_options['max_length'] = 4096      # Maximum tokens to generate
search_options['temperature'] = 0.00001  # Very low temperature for deterministic output
search_options['top_p'] = 1.0            # Nucleus sampling parameter
search_options['do_sample'] = False       # Disable sampling for consistent results

## Krok 4: Definovanie dostupných funkcií

Tu definujeme funkcie, ktoré môže náš AI asistent používať. V tomto príklade máme dve funkcie súvisiace s cestovaním:
1. **booking_flight_tickets**: Na rezerváciu letov medzi letiskami
2. **booking_hotels**: Na rezerváciu hotelového ubytovania

Definície funkcií nasledujú formát schémy volania funkcií od OpenAI.


In [8]:
tool_list = '[{"name": "booking_flight_tickets", "description": "booking flights", "parameters": {"origin_airport_code": {"description": "The name of Departure airport code", "type": "string"}, "destination_airport_code": {"description": "The name of Destination airport code", "type": "string"}, "departure_date": {"description": "The date of outbound flight", "type": "string"}, "return_date": {"description": "The date of return flight", "type": "string"}}}, {"name": "booking_hotels", "description": "booking hotel", "parameters": {"destination": {"description": "The name of the city", "type": "string"}, "check_in_date": {"description": "The date of check in", "type": "string"}, "checkout_date": {"description": "The date of check out", "type": "string"}}}]'

## Krok 5: Pomocné funkcie na generovanie gramatiky

Tieto pomocné funkcie prevádzajú naše definície funkcií do formátu gramatiky Lark, ktorý používa ONNX Runtime GenAI na riadené generovanie. To zaručuje, že model vytvára platné volania funkcií v správnom formáte JSON.


In [9]:
def get_lark_grammar(input_tools):
    tools_list = get_tools_list(input_tools)
    prompt_tool_input = create_prompt_tool_input(tools_list)
    if len(tools_list) == 1:
        # output = ("start: TEXT | fun_call\n" "TEXT: /[^{](.|\\n)*/\n" " fun_call: <|tool_call|> %json " + json.dumps(tools_list[0]))
        output = ("start: TEXT | fun_call\n" "TEXT: /[^{](.|\\n)*/\n" " fun_call: <|tool_call|> %json " + json.dumps(convert_tool_to_grammar_input(tools_list[0])))
        return prompt_tool_input, output
    else:
        return prompt_tool_input, "start: TEXT | fun_call \n TEXT: /[^{](.|\n)*/ \n fun_call: <|tool_call|> %json {\"anyOf\": [" + ','.join([json.dumps(tool) for tool in tools_list]) + "]}"


In [10]:
def get_tools_list(input_tools):
    # input_tools format: '[{"name": "fn1", "description": "fn details", "parameters": {"p1": {"description": "details", "type": "string"}}},
    # {"fn2": 2},{"fn3": 3}]'
    tools_list = []
    try:
        tools_list = json.loads(input_tools)
    except json.JSONDecodeError:
        raise ValueError("Invalid JSON format for tools list, expected format: '[{\"name\": \"fn1\"},{\"name\": \"fn2\"}]'")
    if len(tools_list) == 0:
        raise ValueError("Tools list cannot be empty")
    return tools_list

In [11]:
def create_prompt_tool_input(tools_list):
    tool_input = str(tools_list[0])
    for tool in tools_list[1:]:
        tool_input += ',' + str(tool)
    return tool_input

In [12]:
def convert_tool_to_grammar_input(tool):
    param_props = {}
    required_params = []
    for param_name, param_info in tool.get("parameters", {}).items():
        param_props[param_name] = {
            "type": param_info.get("type", "string"),
            "description": param_info.get("description", "")
        }
        required_params.append(param_name)
    output_schema = {
        "description": tool.get('description', ''),
        "type": "object",
        "required": ["name", "parameters"],
        "additionalProperties": False,
        "properties": {
            "name": { "const": tool["name"] },
            "parameters": {
                "type": "object",
                "properties": param_props,
                "required": required_params,
                "additionalProperties": False
            }
        }
    }
    if len(param_props) == 0:
        output_schema["required"] = ["name"]
    return output_schema

In [13]:
get_lark_grammar(tool_list)

("{'name': 'booking_flight_tickets', 'description': 'booking flights', 'parameters': {'origin_airport_code': {'description': 'The name of Departure airport code', 'type': 'string'}, 'destination_airport_code': {'description': 'The name of Destination airport code', 'type': 'string'}, 'departure_date': {'description': 'The date of outbound flight', 'type': 'string'}, 'return_date': {'description': 'The date of return flight', 'type': 'string'}}},{'name': 'booking_hotels', 'description': 'booking hotel', 'parameters': {'destination': {'description': 'The name of the city', 'type': 'string'}, 'check_in_date': {'description': 'The date of check in', 'type': 'string'}, 'checkout_date': {'description': 'The date of check out', 'type': 'string'}}}",
 'start: TEXT | fun_call \n TEXT: /[^{](.|\n)*/ \n fun_call: <|tool_call|> %json {"anyOf": [{"name": "booking_flight_tickets", "description": "booking flights", "parameters": {"origin_airport_code": {"description": "The name of Departure airport c

## Krok 6: Otestujte generovanie gramatiky

Otestujme naše funkcie na generovanie gramatiky, aby sme zistili, ako prevádzajú definície nástrojov do správneho formátu.


In [14]:
prompt_tool_input, guidance_input = get_lark_grammar(tool_list)

## Krok 7: Pripravte systémovú výzvu a generátor

Teraz vytvoríme systémovú výzvu, ktorá modelu oznámi dostupné nástroje, a nastavíme generátor s parametrami riadeného generovania.


In [15]:
# Define the system prompt that introduces the assistant and its capabilities
system_prompt = "You are a helpful assistant with these tools."

In [16]:
# Format the system message with tools information
messages = f"""[{{"role": "system", "content": "{system_prompt}", "tools": "{prompt_tool_input}"}}]"""

In [17]:
# Apply chat template to format the system prompt properly
tokenizer_input_system_prompt = tokenizer.apply_chat_template(messages=messages, add_generation_prompt=False)

In [18]:
tokenizer_input_system_prompt

"<|system|>You are a helpful assistant with these tools.<|tool|>{'name': 'booking_flight_tickets', 'description': 'booking flights', 'parameters': {'origin_airport_code': {'description': 'The name of Departure airport code', 'type': 'string'}, 'destination_airport_code': {'description': 'The name of Destination airport code', 'type': 'string'}, 'departure_date': {'description': 'The date of outbound flight', 'type': 'string'}, 'return_date': {'description': 'The date of return flight', 'type': 'string'}}},{'name': 'booking_hotels', 'description': 'booking hotel', 'parameters': {'destination': {'description': 'The name of the city', 'type': 'string'}, 'check_in_date': {'description': 'The date of check in', 'type': 'string'}, 'checkout_date': {'description': 'The date of check out', 'type': 'string'}}}<|/tool|><|end|><|endoftext|>"

In [19]:
input_tokens = tokenizer.encode(tokenizer_input_system_prompt)

In [20]:
input_tokens = input_tokens[:-1]

In [21]:
system_prompt_length = len(input_tokens)

## Krok 8: Inicializácia generátora s riadeným generovaním

Teraz vytvoríme generátor s našimi nakonfigurovanými parametrami a použijeme gramatiku Lark na riadené generovanie.


In [22]:
# Create generator parameters and apply search options
params = og.GeneratorParams(model)
params.set_search_options(**search_options)

In [23]:
# Apply Lark grammar for guided generation to ensure valid function call format
params.set_guidance("lark_grammar", guidance_input)

In [24]:
generator = og.Generator(model, params)

In [25]:
generator.append_tokens(input_tokens)

## Krok 9: Otestujte paralelné volanie funkcií

Teraz otestujme naše nastavenie na zložitejšom scenári rezervácie cestovania, ktorý vyžaduje volanie viacerých funkcií.


In [26]:
# Complex travel booking request that requires both flight and hotel booking
text = "book flight ticket from Beijing to Paris(using airport code) in 2025-12-04 to 2025-12-10 , then book hotel from 2025-12-04 to 2025-12-10 in Paris"

In [27]:
# Format user message and apply chat template
messages = f"""[{{"role": "user", "content": "{text}"}}]"""

# Apply Chat Template for user input
user_prompt = tokenizer.apply_chat_template(messages=messages, add_generation_prompt=True)
input_tokens = tokenizer.encode(user_prompt)
generator.append_tokens(input_tokens)

In [28]:
user_prompt

'<|user|>book flight ticket from Beijing to Paris(using airport code) in 2025-12-04 to 2025-12-10 , then book hotel from 2025-12-04 to 2025-12-10 in Paris<|end|><|assistant|>'

### Generovanie volaní funkcií

Model teraz bude generovať štruktúrované volania funkcií na základe požiadavky používateľa. Vďaka riadenému generovaniu bude výstup vo formáte platného JSON, ktorý je možné priamo vykonať.

**Očakávaný výstup**: Model by mal generovať volania funkcií pre:
1. `booking_flight_tickets` s detailmi letu z Pekingu (PEK) do Paríža (CDG)
2. `booking_hotels` s detailmi ubytovania v Paríži

Spustite bunku nižšie, aby ste videli živé generovanie:


In [29]:
# Generate tokens one by one and stream the output
while not generator.is_done():
    generator.generate_next_token()
    new_token = generator.get_next_tokens()[0]
    print(tokenizer_stream.decode(new_token), end='', flush=True)

[{"name": "booking_flight_tickets", "arguments": {"origin_airport_code": "PEKK", "destination_airport_code": "CDG", "departure_date": "2025-12-04", "return_date": "2025-12-10"}}, {"name": "booking_hotels", "arguments": {"destination": "Paris", "check_in_date": "2025-12-04", "checkout_date": "2025-12-10"}}]

## Záver

🎉 **Gratulujeme!** Úspešne ste implementovali paralelné volanie funkcií s Phi-4 Mini pomocou ONNX Runtime GenAI.

### Čo ste sa naučili:

1. **Nastavenie modelu**: Ako nakonfigurovať Phi-4 Mini s ONNX Runtime GenAI
2. **Definícia funkcií**: Ako definovať schémy funkcií pre volanie AI funkcií
3. **Riadená generácia**: Ako použiť gramatiku Lark na generovanie štruktúrovaného výstupu
4. **Paralelné volanie funkcií**: Ako spracovať komplexné požiadavky vyžadujúce viacnásobné volanie funkcií

### Hlavné výhody:

- ✅ **Štruktúrovaný výstup**: Riadená generácia zabezpečuje platné JSON volania funkcií
- ✅ **Paralelné spracovanie**: Možnosť spracovať viacnásobné volania funkcií v jednej požiadavke
- ✅ **Vysoký výkon**: ONNX Runtime poskytuje optimalizované inferencie
- ✅ **Flexibilná schéma**: Jednoduché pridávanie alebo úprava definícií funkcií

### Zdroje:

- [Phi-4 Mini Dokumentácia](https://huggingface.co/microsoft/Phi-4-mini-onnx)
- [ONNX Runtime GenAI Dokumentácia](https://onnxruntime.ai/docs/genai/)
- [Najlepšie praktiky volania funkcií](https://platform.openai.com/docs/guides/function-calling)



---

**Upozornenie**:  
Tento dokument bol preložený pomocou služby na automatický preklad [Co-op Translator](https://github.com/Azure/co-op-translator). Hoci sa snažíme o presnosť, upozorňujeme, že automatické preklady môžu obsahovať chyby alebo nepresnosti. Pôvodný dokument v jeho pôvodnom jazyku by mal byť považovaný za autoritatívny zdroj. Pre dôležité informácie odporúčame profesionálny ľudský preklad. Nezodpovedáme za akékoľvek nedorozumenia alebo nesprávne interpretácie vyplývajúce z použitia tohto prekladu.
